In [ ]:
import geopandas as gpd
import pandas as pd
import glob
import numpy as np
import os
from helper_functions import prepare_gadm_layers, process_reforestation

In [ ]:
reforestation_gdf = gpd.read_parquet("../midsave/new_updated_reforestation_projects.parquet")

In [ ]:
reforestation_gdf = reforestation_gdf.set_crs(epsg=4326)

In [ ]:
reforestation_gdf["site_id_created"]

### Checking cases where polygon is an exact administration area


In [ ]:


# Defining data File paths
GADM_PATH = "../input/gadm_410-levels.gpkg"
REFORESTATION_PATH = "../midsave/new_updated_reforestation_projects.parquet"
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

gadm_data = prepare_gadm_layers(GADM_PATH)

# Processing the geometry reforestation data
final_gdf = process_reforestation(REFORESTATION_PATH, gadm_data)

output_path = os.path.join(OUTPUT_DIR, "processed_reforestation.gpkg")
final_gdf.to_file(output_path, driver="GPKG")
print(f"Results saved to {output_path}")

In [ ]:
reforestation_gdf["exact_admin_area"] = reforestation_gdf["site_id_created"].isin(final_gdf["site_id_created"]).map({True: "Yes", False: "No"})


In [ ]:
reforestation_gdf["exact_admin_area"].value_counts()

In [ ]:
reforestation_gdf.info()

### Checking cases where polygon is a 95% perfect circle

In [ ]:




reforestation_gdf["circularity"] = (4 * np.pi * reforestation_gdf.area) / (reforestation_gdf.length ** 2)


circle_threshold = 0.95


reforestation_gdf["polygon_acircle_oval_95"] = np.where(
   reforestation_gdf["circularity"] > circle_threshold, "Yes", "No"
)


reforestation_gdf["intersecting_circle"] = "No"


circle_oval_gdf = reforestation_gdf[reforestation_gdf["polygon_acircle_oval_95"] == "Yes"]


for i, circle1 in circle_oval_gdf.iterrows():
    for j, circle2 in circle_oval_gdf.iterrows():
        if i >= j:
            continue  

       
        if circle1.geometry.intersects(circle2.geometry) or circle1.geometry.within(circle2.geometry) or circle2.geometry.within(circle1.geometry):
            
            reforestation_gdf.at[i, "intersecting_circle"] = "Yes"
            reforestation_gdf.at[j, "intersecting_circle"] = "Yes"


print(reforestation_gdf.info())
# print(reforestation_gdf)

In [ ]:
reforestation_gdf["circularity"].describe()

In [ ]:

polygon_counts = reforestation_gdf["polygon_acircle_oval"].value_counts()

intersecting_counts = reforestation_gdf["intersecting_circle"].value_counts()

print("Counts for 'polygon_acircle_oval':")
print(polygon_counts)

print("\nCounts for 'intersecting_circle':")
print(intersecting_counts)

In [ ]:
reforestation_gdf.info()

In [ ]:
reforestation_gdf.drop(['geometry_type'], axis=1, inplace=True)

reforestation_gdf.info()

In [ ]:
reforestation_gdf.to_parquet("../midsave/newest_with_circularity_exact_site_updated_reforestation_projects.parquet")